<a href="https://colab.research.google.com/github/gyesibiney/Career-Accelerator-LP5-Sentinent-Analysis/blob/main/DistilBERT_base_uncased_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00


### Importing Libraries

In [ ]:
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#import emoji
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

from matplotlib import axes
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import re,string, unicodedata
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk.corpus import wordnet
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import string

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# from transformers import DistilBertTokenizer, DistilBertModel
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")
#text = "Replace me by any text you'd like."
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)


# from transformers import AutoModelForSequenceClassification
# from transformers import TFAutoModelForSequenceClassification
# from transformers import AutoTokenizer
# import numpy as np
# from scipy.special import softmax
import csv
# import urllib.request

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Push_to_HuggingFace

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Loading google drive

In [ ]:
%%time
df = pd.read_csv('/content/drive/MyDrive/Distil-bert/df.csv')
df.tail(10)

CPU times: user 48.2 ms, sys: 4.92 ms, total: 53.2 ms
Wall time: 300 ms


,label,agreement,clean_tweet
9989,1.0,1.000000,"[('stop', 'VB'), ('worrying', 'NN'), ('theyll'..."
9990,0.0,1.000000,"[('dawg', 'NN'), ('said', 'VBD'), ('gone', 'VB..."
9991,0.0,1.000000,"[('one', 'CD'), ('half', 'NN'), ('hour', 'NN')..."
9992,1.0,0.666667,"[('doesnt', 'NN'), ('want', 'VBP'), ('vaccinat..."
9993,1.0,1.000000,"[('ill', 'NNS'), ('say', 'VBP'), ('glad', 'JJ'..."
9994,1.0,1.000000,"[('living', 'NN'), ('time', 'NN'), ('sperm', '..."
9995,1.0,0.666667,"[('spite', 'JJ'), ('measles', 'NNS'), ('outbre..."
9996,0.0,1.000000,"[('interesting', 'VBG'), ('trend', 'NN'), ('ch..."
9997,0.0,1.000000,"[('cdc', 'NNS'), ('say', 'VBP'), ('measles', '..."
9998,1.0,0.666667,"[('pneumonia', 'NN'), ('vaccine', 'NN'), ('wom..."


I manually split the training set to have a training subset ( a dataset the model will learn on), and an evaluation subset ( a dataset the model with use to compute metric scores to help use to avoid some training problems like the overfitting one ).

There are multiple ways to do split the dataset. You'll see two commented line showing you another one.

In [ ]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
train.head()

,label,agreement,clean_tweet
9303,0.0,1.000000,"[('mickey', 'NN'), ('measles', 'VBZ'), ('gone'..."
3907,0.0,1.000000,"[('s1256', 'JJ'), ('new', 'JJ'), ('extends', '..."
795,1.0,0.666667,"[('ignorance', 'NN'), ('vaccine', 'NN'), ('isn..."
5791,0.0,1.000000,"[('pakistan', 'NN'), ('partly', 'RB'), ('suspe..."
3431,0.0,1.000000,"[('news', 'NN'), ('ive', 'JJ'), ('gone', 'VBN'..."


In [ ]:
eval.head()

,label,agreement,clean_tweet
6569,1.0,1.000000,"[('childrens', 'NNS'), ('museum', 'VBP'), ('ho..."
1754,1.0,1.000000,"[('properly', 'RB'), ('immunized', 'VBN'), ('p..."
3325,1.0,1.000000,"[('thx', 'NN'), ('posting', 'VBG'), ('vaccinat..."
1485,1.0,0.666667,"[('baby', 'NN'), ('exactly', 'RB'), ('everyone..."
4175,1.0,1.000000,"[('meeting', 'VBG'), ('tonight', 'VBD'), ('830..."


In [ ]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (7999, 3), eval is (2000, 3)


In [ ]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/Distil-bert/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/Distil-bert/eval_subset.csv", index=False)


In [ ]:
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/Distil-bert/train_subset.csv',
                        'eval': '/content/drive/MyDrive/Distil-bert/eval_subset.csv'}, encoding = "ISO-8859-1")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c406be8641e0fbd4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['clean_tweet'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = [ 'label','clean_tweet','agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7999
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7999
})

In [ ]:
from transformers import TrainingArguments

# Configure the trianing parameters like `num_train_epochs`:
# the number of time the model will repeat the training loop over the dataset
#training_args = TrainingArguments("test_trainer", num_train_epochs=3000, load_best_model_at_end=True,)
training_args = TrainingArguments("test_trainer", num_train_epochs=3, load_best_model_at_end=True,evaluation_strategy="steps",save_strategy="steps",push_to_hub=True,)

In [ ]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.

In [ ]:
train_dataset = dataset['train'].shuffle(seed=10) #.select(range(40000)) # to select a part
eval_dataset = dataset['eval'].shuffle(seed=10)

## other way to split the train set ... in the range you must use:
# # int(num_rows*.8 ) for [0 - 80%] and  int(num_rows*.8 ),num_rows for the 20% ([80 - 100%])
# train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
# eval_dataset = dataset['train'].shuffle(seed=10).select(range(40000, 41000))

In [ ]:
from transformers import Trainer, TrainingArguments

# Set up your training arguments
training_args = TrainingArguments(
    output_dir="./output",        # Output directory for checkpoints and logs
    num_train_epochs=3,           # Number of training epochs
    per_device_train_batch_size=8, # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    learning_rate=2e-5,            # Initial learning rate
    weight_decay=0.01,             # Weight decay (L2 regularization)
    logging_dir="./logs",         # Directory for storing logs
    logging_steps=500,            # Log metrics every n steps
    evaluation_strategy="steps",  # Evaluation strategy ("steps", "epoch")
    eval_steps=1000,              # Evaluate every n steps
    save_strategy="steps",        # Model checkpoint save strategy ("steps", "epoch")
    save_steps=1000,              # Save checkpoint every n steps
    warmup_ratio=0.1,             # Percentage of total training steps for warmup
    gradient_accumulation_steps=1, # Number of steps to accumulate gradients
    #fp16=True,                    # Use mixed precision training (requires compatible GPU)
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset,
)

In [ ]:
#trainer.push_to_hub()

Cloning https://huggingface.co/Sonny4Sonnix/output into local empty directory.


Download file pytorch_model.bin:   0%|          | 6.25k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.75k/3.75k [00:00<?, ?B/s]

Clean file training_args.bin:  27%|##6       | 1.00k/3.75k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.75k [00:00<?, ?B/s]

To https://huggingface.co/Sonny4Sonnix/output
   1e91cef..79dbf88  main -> main

   1e91cef..79dbf88  main -> main



'https://huggingface.co/Sonny4Sonnix/output/commit/79dbf88b84b559917d5733e45cc9ddd63ca8cee9'

In [ ]:
# Launch the learning process: training
trainer.train()

Step,Training Loss,Validation Loss
1000,0.699700,0.704416
2000,0.636500,0.631700
3000,0.548400,0.647590


TrainOutput(global_step=3000, training_loss=0.6588027445475261, metrics={'train_runtime': 1224.2963, 'train_samples_per_second': 19.601, 'train_steps_per_second': 2.45, 'total_flos': 3178876855292928.0, 'train_loss': 0.6588027445475261, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

To https://huggingface.co/Sonny4Sonnix/output
   79dbf88..0ccccf6  main -> main

   79dbf88..0ccccf6  main -> main

To https://huggingface.co/Sonny4Sonnix/output
   0ccccf6..fd04f45  main -> main

   0ccccf6..fd04f45  main -> main



'https://huggingface.co/Sonny4Sonnix/output/commit/0ccccf6459bac9723af3744cd6d86070cb8dfc52'

*Don*'t worry the above issue, it is a `KeyboardInterrupt` that means I stopped the training to avoid taking a long time to finish.

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Launch the final evaluation
trainer.evaluate()

{'eval_loss': 0.6475904583930969,
 'eval_runtime': 31.1132,
 'eval_samples_per_second': 64.281,
 'eval_steps_per_second': 8.035,
 'epoch': 3.0}

In [ ]:
# Access the training logs
training_logs = trainer.state.log_history

In [ ]:
#  Retrieve the relevant information
# optimizer_lr = training_logs["learning_rate"]  # Optimizer learning rate
# eval_loss = training_logs["eval_loss"]  # Evaluation loss
# eval_runtime = training_logs["eval_runtime"]  # Evaluation runtime
# learning_rate_decay = training_logs["learning_rate_decay"]  # Learning rate decay
# eval_samples_per_second = training_logs["eval_samples_per_second"]  # Evaluation samples per second
# eval_steps_per_second = training_logs["eval_steps_per_second"]  # Evaluation steps per second